# Yee Algoritmus

Yee algoritnmus je pomenovaný po matematikovi [Kane Yee](https://en.wikipedia.org/wiki/Finite-difference_time-domain_method), ktorý ho predstavil v roku 1966. Všeobecný postup výpočtu elektromagnetických polí je možné defnovať nasledujúcimi krokmi

- vo zvolenom priestore definujeme vybrané uzly, v ktorých budeme určovať hodnoty polí $E$ a $H$ 
- aproximujeme časové a priestorové derivácie v Ampérovom a Faradayovom zákone konečnými diferenciami 
- vyriešime sústavu diferenčných rovníc tak, aby sme získali *aktualizačné* rovnice, ktoré popisujú budúce hodnoty na základe predchádzajúcich hodnôt v priestore a čase
- v jednotlivých časových krokoch spočítame hodnoty polí $E$,$H$ pre všetky uzly priestoru
- opakujeme predchádzajúci krok, kým nedosiahneme stanovený čas ukončenia simulácie

Vyššie uvedený postup ukážeme na odvodení algoritmu pre jednorozmerný prípad.

## Yee Algoritmus v 1D 

Predpokladajme jednorozmenrný priestor v ktorom sa môže šíríť elektromagnetická vlna v smere osi $x$. Zložky elektromagnetického poľa sú na seba kolmé, predpokladajme, že elektrická zložka je orintovaná v smere osi $z$ a magnetická v smere osi $y$.

Podľa Faradayovho zákona potom môžeme písať

\begin{equation}
-\mu \frac{\partial \mathbf{H}}{\partial t} = \nabla \times \mathbf{E} = 
\begin{vmatrix} \mathbf{i} & \mathbf{j} & \mathbf{k} \\
{\dfrac{\partial}{\partial x}} & 0 & 0 \\
0 & 0 & E_z \end{vmatrix} =
-\mathbf{j} \frac{\partial E_z}{\partial x}
\end{equation}

a z Ampérovho zákona vyplýva

\begin{equation}
-\epsilon \frac{\partial \mathbf{E}}{\partial t} = \nabla \times \mathbf{H} = 
\begin{vmatrix} \mathbf{i} & \mathbf{j} & \mathbf{k} \\
{\dfrac{\partial}{\partial x}} & 0 & 0 \\
0 & H_y & 0 \end{vmatrix} =
-\mathbf{k} \frac{\partial H_y}{\partial x}
\end{equation}

po rozpísaní na zložky dostaneme sústavu skalárnych rovníc

\begin{align}
\mu \frac{\partial H_y}{\partial t}  &= \frac{\partial E_z}{\partial x} \\
\epsilon \frac{\partial E_z}{\partial t}  &= \frac{\partial H_y}{\partial x}
\end{align}

Z rovníc je zrejmé, že časová derivácia jedného poľa závisí od priestorovej derivácia druhého poľa. 

## Diskretizácia rovníc

Pre disktertizáciu rovníc zavedieme označenie, ktoré popisuje časovo-priestorovú lokalizáciu polí, index $q$ nie je mocnina, ale časový krok

\begin{align*}
E_z(x,t) &= E_z(m \, \Delta x, q \, \Delta t) = E_z^q[m] \\
H_y(x,t) &= H_y(m \, \Delta x, q \, \Delta t) = H_y^q[m] \\
\end{align*}


### Diskretizácia Faradayovho zákona

V priestorovo-časovej súradnici $[(m+1/2)\Delta x, q \Delta t]$ môžeme pre Faradayov zákon písať 

\begin{equation*}
\mu \frac{\partial H_y}{\partial t} \biggr\rvert_{(m+1/2)\Delta x, \, q \Delta t}  = 
\frac{\partial E_z}{\partial x} \biggr\rvert_{(m+1/2)\Delta x, \, q \Delta t}
\end{equation*} 

Deriváciu $H_y$ v čase $q \Delta t$ aproximovať pomocou dvoch po sebe nasledujúcich hodnôt

\begin{equation*}
\mu \frac{\partial H_y}{\partial t} \biggr\rvert_{(m+1/2)\Delta x, \, q \Delta t}  \approx \mu \frac{H_y^{q+1/2}[m+1/2] - H_y^{q-1/2}[m+1/2]}{\Delta t}
\end{equation*}

Deriváciu $E_z$ na pozícii $(m+1/2)\Delta x$  aproximovať pomocou dvoch susedných hodnôt

\begin{equation*}
\frac{\partial E_z}{\partial x} \biggr\rvert_{(m+1/2)\Delta x, \, q \Delta t} \approx \frac{E_z^{q}[m+1] - E_z^{q}[m] }{\Delta x}
\end{equation*}

Z vyššie uvedeného potom vyplýva

\begin{equation*}
\mu \frac{H_y^{q+1/2}[m+1/2] - H_y^{q-1/2}[m+1/2]}{\Delta t} = \frac{E_z^{q}[m+1] - E_z^{q}[m] }{\Delta x}
\end{equation*}

po úprave dostaneme pre hodnotu $H_y^{q+1/2}[m+1/2]$ *aktualizačnú* rovnicu v tvare 

\begin{equation}
H_y^{q+1/2}[m+1/2] = H_y^{q-1/2}[m+1/2] + \frac{\Delta t}{\mu \, \Delta x} \big( E_z^{q}[m+1] - E_z^{q}[m] \big)
\end{equation}



Priestorovo-časové rozloženie zložiek poľa $E_z$ a $H_y$ potom môžeme znázorniť na nasledujúcom diagrame, $\otimes$ označuje bod $[(m+1/2)\Delta x, q \Delta t]$.

<img src="./img/fdtd_01.png" width=600px alt="Aktualizačná rovnica pre Faradayov zákon" scale="0.8"/>

Je zrejmé, že budúca hodnota magnetického poľa závisí od predchádzajúcej hodnoty a susedných hodnôt elektrického poľa.

### Diskretizácia Ampérovho zákona

V priestorovo-časovej súradnici $[(m \Delta x, (q+1/2) \Delta t]$ môžeme pre Ampérov zákon písať 

\begin{equation*}
\epsilon \frac{\partial E_z}{\partial t} \biggr\rvert_{m \Delta x, \, (q+1/2) \Delta t}  = 
\frac{\partial H_y}{\partial x} \biggr\rvert_{m \Delta x, \, (q+1/2) \Delta t}
\end{equation*} 

Z vyššie uvedeného potom vyplýva

\begin{equation*}
\epsilon \frac{E_z^{q+1}[m] - E_z^{q}[m]}{\Delta t} = \frac{H_y^{q + 1/2}[m+1/2] - H_y^{q+1/2}[m-1/2] }{\Delta x}
\end{equation*}

po úprave dostaneme pre hodnotu $E_z^{q+1}[m]$ aktualizačnú rovnicu v tvare 

\begin{equation}
E_z^{q+1}[m] = E_z^{q}[m] + \frac{\Delta t}{\epsilon \, \Delta x} \big({H_y^{q + 1/2}[m + 1/2] - H_y^{q+1/2}[m-1/2] } \big)
\end{equation}


<img src="./img/fdtd_02.png" width=600px alt="Aktualizačná rovnica pre Ampérov zákon" scale="0.8"/>

Podobne ako pri diskretizovanom Faradayovom zákone budúca hodnota elektrického poľa závisí od predchádzajúcej hodnoty a susedných hodnôt magnetického poľa.


## Courantovo číslo

Koeficienty v aktualizačných rovniciach $\Delta t\, / \, \mu \Delta x$ a $\Delta t\, / \, \epsilon \Delta x$ určujú, ako ďaleko (v čase a priestore) sa môže šíriť elektromagnetická energia. Ich veľkosť má vplyv na stabiliu riešenia a zároveň určujú rozmerové paramere simulačného modelu. Pri platnosti vzťahov

\begin{align*}
c &= \frac{1}{\sqrt{\epsilon_0 \mu_0}} \\
z_0 &= \sqrt{\frac{\mu_0}{\epsilon_0}} \\
\mu &= \mu_0 \mu_r \\
\epsilon &= \epsilon_0 \epsilon_r
\end{align*}

môžeme koeficienty prepísať do tvaru

\begin{equation*}
\frac{\Delta t}{\epsilon \Delta x} = 
\frac{1}{\epsilon_0 \epsilon_r} \frac{ \sqrt{\epsilon_0 \mu_0}}{ \sqrt{\epsilon_0 \mu_0}} \frac{\Delta t}{\Delta x} =
\frac{\sqrt{\epsilon_0 \mu_0}}{\epsilon_0 \epsilon_r} \frac{c \Delta t}{\Delta x} = 
\frac{1}{\epsilon_r} \sqrt{\frac{\mu_0}{\epsilon_0}} \frac{c \Delta t}{\Delta x} = \frac{z_0}{\epsilon_r} S_c
\end{equation*}

\begin{equation*}
\frac{\Delta t}{\mu \Delta x} = 
\frac{1}{\mu_0 \mu_r} \frac{ \sqrt{\epsilon_0 \mu_0}}{ \sqrt{\epsilon_0 \mu_0}} \frac{\Delta t}{\Delta x} =
\frac{\sqrt{\epsilon_0 \mu_0}}{\mu_0 \mu_r} \frac{c \Delta t}{\Delta x} = 
\frac{1}{\mu_r} \sqrt{\frac{\epsilon_0}{\mu_0}} \frac{c \Delta t}{\Delta x} = 
\frac{1}{\mu_r \, z_0} S_c
\end{equation*}

kde $S_c$ je veličina nazývaná Courantovo číslo, definované ako 

\begin{equation*}
S_c = \frac{c \Delta t}{\Delta x} 
\end{equation*}

**Výber hodnoty $S_c$**

Elektromagnetická vlna sa vo voľnom priestore nemôže šíriť rýchlosťou väčšou ako je rýchlosť svetla $c$. Čas potrebný na prechod vlny o veľkosť jednej bunky vyžaduje minimálny čas $\Delta t = \Delta x \, / \, c$. V dvojrozmernom prípade, ak sa vlna šíri po diagonále, bude požiadavka na hodnotu časového kroku rovná 

\begin{equation*}
\Delta t = \frac{\Delta x}{\sqrt{2} \, c}
\end{equation*}

Pre trojrozmerný prípad pri šírení sa po priestorovej diagonále podobne platí

\begin{equation*}
\Delta t = \frac{\Delta x}{\sqrt{3} \, c}
\end{equation*}

Zovšeobecnením dostaneme *Courantovu podmienku*

\begin{equation*}
\Delta t \leq \frac{\Delta x}{\sqrt{n} \, c}
\end{equation*}

ktorá prakticky hovorí o tom, že (numericky) by mal byť krok v čase rovný alebo menší ako je priestorový krok. Menšia hodnota $\Delta t$ zodpovedá väčšiemu počtu krokov na zvolený interval simulácie, jemnejšie časové rozlíšenie a väčšiu stabilitu riešenia za cenu dlhšieho výpočtu.

V metóde FDTD z praktických dôvodov pokladáme hodnotu $c = 1$ a ako východziu hodnotu pri simulácii zvyčajne volíme $S_c = 1$.